## import statements

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [ ]:
def show_image(arr):
    # 0 is the index of the training image you want to display
    two_d = (np.reshape(arr, (128, 128)) * 255).astype(np.uint8)
    plt.imshow(two_d, cmap='gray', interpolation='nearest')
    plt.show()


In [ ]:
# load files
train_images = np.load('../input/mais-202-fall-2020-kaggle-competition/train_x.npy')
train_labels = pd.read_csv('../input/mais-202-fall-2020-kaggle-competition/train_y.csv')

In [ ]:
# Noise reduction
train_images = np.where(train_images>250, 1, 0)

In [ ]:
# train/validation split
train_X = train_images[:30000]
vld_X = train_images[30000:35000]
test_X = train_images[35000:40000]\

temp = train_labels.Label.to_numpy()
train_Y = temp[:30000]
vld_Y = temp[30000:35000]
test_Y = temp[35000:]

In [ ]:
show_image(train_images[5])

In [ ]:
# save input image dimensions
img_rows, img_cols = 128, 128

# reshaping required for keras API
train_X = train_X.reshape(train_X.shape[0], img_rows, img_cols, 1)
vld_X = vld_X.reshape(vld_X.shape[0], img_rows, img_cols, 1)
test_X = test_X.reshape(test_X.shape[0], img_rows, img_cols, 1)

In [ ]:
train_X.shape

## CNN model building 

In [ ]:

def build_model():
    img_rows, img_cols = 128, 128

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size=(5,5), activation='relu',\
                 input_shape=(img_rows, img_cols,1), padding='same'))
    
    model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    # flatten
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',\
                  optimizer='adam', metrics=['accuracy'])
    
    return model


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#Augmentation-Increasing our dataset by modifying the images to get new ones to avoid overfitting
datagen = ImageDataGenerator(

    rotation_range=20, 
    width_shift_range=0.1,
    shear_range=0.2,
    height_shift_range=0.1,
    zoom_range=0.2
    )
datagen.fit(train_X)

In [ ]:
def fit_model(model, batch_size, epochs):
    history =  model.fit_generator(datagen.flow(train_X, train_Y,
              batch_size=batch_size),
              epochs=epochs,
              verbose=1,
              validation_data=(vld_X, vld_Y))
    return history

In [ ]:
def evaluate(model):
    score = model.evaluate(test_X, test_Y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [ ]:
model_5lay = build_model()

In [ ]:
history = fit_model(model_5lay, 256, 10)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
evaluate(model_5lay)